In [1]:
import joblib
import pandas as pd
import numpy as np

In [2]:
import sys

MOD_FOLDER = '../'
# setting path to enable import from the parent directory
sys.path.append(MOD_FOLDER)
print(sys.path)

['/cluster/raid/home/himanshu.rawlani/propaganda_detection/prototex_custom_latest/Notebooks', '/cluster/raid/home/himanshu.rawlani/miniconda3/envs/prototex/lib/python39.zip', '/cluster/raid/home/himanshu.rawlani/miniconda3/envs/prototex/lib/python3.9', '/cluster/raid/home/himanshu.rawlani/miniconda3/envs/prototex/lib/python3.9/lib-dynload', '', '/cluster/raid/home/himanshu.rawlani/miniconda3/envs/prototex/lib/python3.9/site-packages', '../']


In [3]:
from utils import *
from preprocess import *

In [4]:
from transformers import BartTokenizer

In [5]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')


train_df = pd.read_csv("../data/logical_fallacy/edu_train.csv")
dev_df = pd.read_csv("../data/logical_fallacy/edu_dev.csv")
test_df = pd.read_csv("../data/logical_fallacy/edu_test.csv")

train_df = train_df[train_df['updated_label'] != 'equivocation']
dev_df = dev_df[dev_df['updated_label'] != 'equivocation']
test_df = test_df[test_df['updated_label'] != 'equivocation']


train_labels = train_df['updated_label'].tolist()
test_labels = test_df['updated_label'].tolist()

In [6]:
bestk_train_data_per_proto = joblib.load("bestk_train_data_per_proto.joblib")
best_protos_per_testeg = joblib.load("best_protos_per_testeg.joblib")

In [7]:
bestk_train_data_per_proto[0][:10]

array([[1728,  733, 1284, 1569, 1431],
       [ 659, 1216,  212, 1292, 1111],
       [ 468,  130, 1578, 1216, 1480],
       [1612,  300,  260,  271,   66],
       [1557,  941,  352, 1194, 1157],
       [  36, 1597,   87,  402,   55],
       [1444,   20, 1485, 1179,  301],
       [1449, 1363, 1235,   10,  771],
       [1155, 1181,  961,  943,  773],
       [1237,    8, 1627,  683, 1442]])

In [8]:
best_protos_per_testeg[0][:20]

tensor([[ 5, 22,  6, 27,  7],
        [ 5, 26, 27, 16,  4],
        [ 5, 34,  9, 35, 14],
        [ 5, 14, 31, 24, 27],
        [ 5, 22, 24, 15, 27],
        [ 5, 28,  7, 20, 27],
        [ 5, 28,  7, 20,  8],
        [ 5, 25, 12, 27, 16],
        [ 5, 25, 27, 16, 29],
        [ 5, 22, 15, 24, 32],
        [ 5, 14,  1, 31, 17],
        [ 5, 27, 16, 14, 31],
        [ 5, 24, 12, 27,  7],
        [34,  5,  9,  0, 35],
        [ 5, 27, 16, 22, 26],
        [ 5, 22, 16, 27, 11],
        [34,  9,  0, 35,  5],
        [ 5, 23, 27, 16, 34],
        [ 5, 25, 12, 30, 24],
        [ 5, 24, 18, 32, 22]])

In [9]:
results = []
for test_index, test_sample_prototypes in enumerate(best_protos_per_testeg[0]):
    test_sample_label = test_labels[test_index]
    respective_prototypes_train_labels = []
    for prototype in test_sample_prototypes:
        prototype_train_labels = []
        train_samples_close_to_prototype = bestk_train_data_per_proto[0][prototype]
        for train_sample in train_samples_close_to_prototype:
            prototype_train_labels.append(train_labels[train_sample])
        respective_prototypes_train_labels.append(prototype_train_labels)
    results.append((test_sample_label, respective_prototypes_train_labels))

In [10]:
def get_accuracy_of_model_on_label(results, k = 5):
    statistics = []
    for test_label, prototypes_train_labels in results:
        per_test_statistics = []
        for prototype_train_labels in prototypes_train_labels:
            per_test_statistics.append(test_label in prototype_train_labels)
        statistics.append(per_test_statistics)
    statistics = np.array(statistics)
    return np.round(np.mean(np.sum(statistics[:,:k], axis = 1) != 0), 3)

In [11]:
from collections import defaultdict
accuracies = defaultdict(list)

In [12]:
for k in [5, 3, 1]:
    overall_accuracy = get_accuracy_of_model_on_label(results, k)
    accuracies[k].append(overall_accuracy)
    for label in set(train_labels):
        label_specific_accuracy = get_accuracy_of_model_on_label([result for result in results if result[0] == label], k)
        accuracies[k].append(label_specific_accuracy)
        

In [13]:
report_df = pd.DataFrame(
    {
        'class': [
            'overall',
            *list(set(train_labels))
        ],
        'k = 5': accuracies[5],
        'k = 3': accuracies[3],
        'k = 1': accuracies[1]
    }
)

In [14]:
report_df

,class,k = 5,k = 3,k = 1
0,overall,0.861,0.725,0.359
1,false causality,1.000,0.722,0.000
2,appeal to emotion,0.261,0.043,0.000
3,faulty generalization,1.000,1.000,1.000
4,fallacy of logic,0.357,0.143,0.000
5,fallacy of extension,1.000,0.952,0.000
6,circular reasoning,1.000,1.000,1.000
7,fallacy of credibility,0.588,0.471,0.000
8,false dilemma,1.000,0.917,0.000
9,ad populum,1.000,0.533,0.400
